ENTRENAMIENTO Y EVALUACIÓN

In [ ]:
import pandas as pd
import numpy as np
import collections as cols
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#1 Cargar los datos preprocesados
file_path = 'databases/Base de datos para desarrollo v2_dest(preprocesada).csv'
df = pd.read_csv(file_path)

In [3]:
#2 Separar las características de entrada y de salida (objetivo)
input = df.drop(columns=['__destino'])
output = df['__destino']

print(input)
print(output)

        __temperatura   __pulso     __pas     __pad   __sat02
0            0.229959 -0.141229 -0.090837 -0.514129  0.166636
1           -4.667926 -3.667914 -2.134067 -2.059868 -6.671051
2           -4.667926 -1.223280  0.405649  0.495334  0.236409
3            0.229959  1.902645  1.379525  1.094702  0.306181
4            0.203121  0.980898 -0.530036  0.179877  0.236409
...               ...       ...       ...       ...       ...
189077       0.109189  0.099227 -0.186315 -0.514129  0.306181
189078       0.068933  0.540062 -0.186315 -0.514129  0.027092
189079       0.082352 -0.702292  0.653892  0.810791  0.166636
189080       0.162865 -0.501913  0.443840  0.400697  0.236409
189081       0.229959 -1.062976  0.138310 -0.324855  0.306181

[189082 rows x 5 columns]
0         0
1         0
2         0
3         0
4         0
         ..
189077    1
189078    1
189079    1
189080    1
189081    1
Name: __destino, Length: 189082, dtype: int64


In [4]:
# Convertir la salida a categorías
#output = to_categorical(output)
print(output)

0         0
1         0
2         0
3         0
4         0
         ..
189077    1
189078    1
189079    1
189080    1
189081    1
Name: __destino, Length: 189082, dtype: int64


In [5]:
#3 Dividir los datos en conjuntos de entrenamiento 80% y prueba 20% 
input_train, input_test, output_train, output_test = train_test_split(input, output, test_size=0.2, random_state=42)

In [6]:
#4 Construir el modelo dela red neuronal (Perceptron multicapa)
def MLP_NN():
    NumNeurons = 7
    model = Sequential()
    model.add(Dense(64, input_dim=input_train.shape[1]))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Usar 'softmax' para clasificación multiclase

    #opt =  keras.optimizers.Adam(learning_rate=0.001)

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [7]:
#5 Entrenar el modelo
n_epochs = 500
#network = MLP_NN()
#train = network.fit(input_train, output_train, epochs=n_epochs, batch_size=32, validation_split=0.2)


network = MLP_NN()
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
train = network.fit(input_train, output_train, epochs=n_epochs, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/500
3782/3782 [==============================] - 17s 4ms/step - loss: 0.6545 - accuracy: 0.6078 - val_loss: 0.6272 - val_accuracy: 0.6370
Epoch 2/500
3782/3782 [==============================] - 14s 4ms/step - loss: 0.6357 - accuracy: 0.6302 - val_loss: 0.6233 - val_accuracy: 0.6373
Epoch 3/500
3782/3782 [==============================] - 14s 4ms/step - loss: 0.6320 - accuracy: 0.6328 - val_loss: 0.6230 - val_accuracy: 0.6397
Epoch 4/500
3782/3782 [==============================] - 14s 4ms/step - loss: 0.6304 - accuracy: 0.6352 - val_loss: 0.6211 - val_accuracy: 0.6371
Epoch 5/500
3782/3782 [==============================] - 14s 4ms/step - loss: 0.6290 - accuracy: 0.6357 - val_loss: 0.6192 - val_accuracy: 0.6381
Epoch 6/500
3782/3782 [==============================] - 14s 4ms/step - loss: 0.6273 - accuracy: 0.6387 - val_loss: 0.6184 - val_accuracy: 0.6409
Epoch 7/500
3782/3782 [==============================] - 14s 4ms/step - loss: 0.6270 - accuracy: 0.6383 - val_loss: 0.6210 -

In [8]:
#6 Evaluar el modelo
loss, accuracy = network.evaluate(input_test, output_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

1182/1182 [==============================] - 3s 3ms/step - loss: 0.6112 - accuracy: 0.6540
Precisión en el conjunto de prueba: 0.65


In [ ]:
#7 Predicciones
output_pred = network.predict(input_test)
print("Predicciones (probabilidades):")
print(output_pred)

#output_pred_classes = np.argmax(output_pred, axis=1)
#print(output_pred_classes)

# Convertir probabilidades a clases binarias
output_pred_classes = (output_pred > 0.5).astype(int)  # Umbral de 0.5
print("Clases predichas:")
print(output_pred_classes.flatten())  # Asegura que el resultado sea un vector 1D

# Convertir el conjunto de prueba a clases binarias (si es necesario)
#output_test_classes = np.argmax(output_test, axis=1)
output_test_classes = output_test  # Si ya está en formato binario
print("Clases reales:")
print(output_test_classes)

1182/1182 [==============================] - 3s 3ms/step
Predicciones (probabilidades):
[[0.42583144]
 [0.43740487]
 [0.48779157]
 ...
 [0.6191686 ]
 [0.79937065]
 [0.54637444]]
Clases predichas:
[0 0 0 ... 1 1 1]
Clases reales:
35960     0
46028     0
39676     0
169288    1
41981     0
         ..
116898    1
164941    1
117280    1
180830    1
159884    1
Name: __destino, Length: 37817, dtype: int64


In [12]:
#8 Generar el reporte de clasificación
print(classification_report(output_test_classes, output_pred_classes))

              precision    recall  f1-score   support

           0       0.67      0.60      0.63     18942
           1       0.64      0.71      0.67     18875

    accuracy                           0.65     37817
   macro avg       0.66      0.65      0.65     37817
weighted avg       0.66      0.65      0.65     37817



In [13]:
#9 Matriz de confusión
print(confusion_matrix(output_test_classes, output_pred_classes))

[[11312  7630]
 [ 5455 13420]]


In [16]:
# Matriz de confusión con datos de entrenamiento
output_train_pred = network.predict(input_train)
#output_train_pred_classes = np.argmax(output_train_pred, axis=1)
output_train_pred_classes = (output_train_pred > 0.5).astype(int)
#output_train_classes = np.argmax(output_train, axis=1)
output_train_classes = output_train
print(confusion_matrix(output_train_classes, output_train_pred_classes))

4728/4728 [==============================] - 14s 3ms/step
[[45129 30470]
 [22032 53634]]


In [ ]:
# Guardar el modelo y el escalador
network.save('modelo/modelo_entrenado.h5')

/home/manny/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
